In [25]:
# Importing the libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import os

from IPython.display import HTML

import warnings
warnings.filterwarnings('ignore')

In [26]:
# File paths
file_actual = r'../../output_ribasim/ESC-00.csv'

# Read CSVs into dataframes
DF_ACTUAL = pd.read_csv(file_actual,parse_dates=['Fecha'],index_col="Fecha",dayfirst=False,na_values="NA")

In [27]:
DF_ACTUAL.index.name = 'Fecha'  # Set index Fecha
# DF_ACTUAL['year'] = DF_ACTUAL.index.year
# DF_ACTUAL['month'] = DF_ACTUAL.index.month
DF_ACTUAL['monthday'] = DF_ACTUAL.index.day_of_year

In [28]:
HTML(DF_ACTUAL.tail(6).to_html())

,ANO,MES,DIA,CG,PS,80%_CG,80%_PS,TOMA AACC,NO REGULADO,TRASVASE,ARAZATI_1,ARAZATI_2,DEMANDA,monthday
Fecha,,,,,,,,,,,,,,
2025-05-01,2025,5,1,0.0,0.13,0.0,0.11,22.77,22.67,1.27,1.15,0.55,7.63,121
2025-05-11,2025,5,11,0.0,0.13,0.0,0.11,39.34,39.23,1.27,1.15,0.55,7.63,131
2025-05-21,2025,5,21,0.0,3.12,0.0,2.50,35.71,33.21,1.27,1.15,0.55,7.63,141
2025-06-01,2025,6,1,0.0,5.47,0.0,4.37,17.78,13.41,1.27,1.15,0.55,7.41,152
2025-06-11,2025,6,11,0.0,30.80,0.0,24.64,107.23,82.60,1.27,1.15,0.55,7.41,162
2025-06-21,2025,6,21,0.0,9.96,0.0,7.97,50.48,42.51,1.27,1.15,0.55,7.41,172


In [29]:
# Calcular el caudal promedio mensual de cada año para las columnas seleccionadas y crear DF_MES

cols = ['80%_CG', '80%_PS', 'TOMA AACC', 'NO REGULADO', 'TRASVASE', 'ARAZATI_1', 'ARAZATI_2', 'DEMANDA']

# Crear columnas de año y mes
DF_ACTUAL['year'] = DF_ACTUAL.index.year
DF_ACTUAL['month'] = DF_ACTUAL.index.month

# Agrupar por año y mes y calcular el promedio mensual
DF_MES = DF_ACTUAL.groupby(['year', 'month'])[cols].mean().reset_index()

# Agregar columna con el número de días de cada mes y año en DF_MES

DF_MES['dias_mes'] = DF_MES.apply(
    lambda row: pd.Period(year=int(row['year']), month=int(row['month']), freq='M').days_in_month,
    axis=1
)

HTML(DF_MES.tail(48).to_html())

,year,month,80%_CG,80%_PS,TOMA AACC,NO REGULADO,TRASVASE,ARAZATI_1,ARAZATI_2,DEMANDA,dias_mes
402,2021,7,2.746667,16.003333,76.320000,57.576667,1.27,1.15,0.55,7.28,31
403,2021,8,2.500000,15.866667,83.790000,65.423333,1.27,1.15,0.55,7.20,31
404,2021,9,2.090000,21.650000,99.303333,75.566667,1.27,1.15,0.55,7.09,30
405,2021,10,1.186667,6.486667,19.696667,12.030000,1.27,1.15,0.55,7.23,31
406,2021,11,2.510000,2.036667,9.946667,5.400000,1.27,1.15,0.55,7.46,30
407,2021,12,2.446667,2.916667,8.623333,3.263333,1.27,1.15,0.55,7.87,31
408,2022,1,0.030000,21.830000,86.496667,64.636667,1.27,1.15,0.55,8.10,31
409,2022,2,0.583333,4.313333,20.156667,15.253333,1.27,1.15,0.55,8.25,28
410,2022,3,1.170000,4.820000,18.553333,12.566667,1.27,1.15,0.55,8.06,31
411,2022,4,0.820000,4.520000,19.686667,14.350000,1.27,1.15,0.55,7.81,30


In [30]:
# Convertir las columnas seleccionadas de m3/s a hm3 usando el correspondiente dias_mes

cols = ['80%_CG', '80%_PS',  'TOMA AACC','NO REGULADO', 'TRASVASE', 'ARAZATI_1', 'ARAZATI_2', 'DEMANDA']

DF_VOL_MES = DF_MES.copy()
for col in cols:
    # m3/s * segundos/día * días/mes / 1e6 = hm3/mes
    DF_VOL_MES[col] = DF_VOL_MES[col] * 86400 * DF_VOL_MES['dias_mes'] / 1e6

HTML(DF_VOL_MES.head(6).to_html())

,year,month,80%_CG,80%_PS,TOMA AACC,NO REGULADO,TRASVASE,ARAZATI_1,ARAZATI_2,DEMANDA,dias_mes
0,1988,1,0.000000,0.000000,0.000000,0.000000,3.401568,3.08016,1.47312,21.695040,31
1,1988,2,7.959456,8.627616,35.345664,18.766944,3.182112,2.88144,1.37808,20.671200,29
2,1988,3,11.740320,35.774496,327.407616,279.901728,3.401568,3.08016,1.47312,21.587904,31
3,1988,4,6.842880,23.137920,122.575680,92.603520,3.291840,2.98080,1.42560,20.243520,30
4,1988,5,5.874624,9.856512,42.140160,26.409024,3.401568,3.08016,1.47312,20.436192,31
5,1988,6,5.123520,11.543040,41.273280,24.615360,3.291840,2.98080,1.42560,19.206720,30


In [31]:
# Calcular el volumen anual por año hidrológico (Abril a Marzo) y agregar columna WATER_YEAR

cols = ['80%_CG', '80%_PS',  'TOMA AACC','NO REGULADO', 'TRASVASE', 'ARAZATI_1', 'ARAZATI_2', 'DEMANDA']

# Definir el año hidrológico: si el mes >= 4, es el año actual, si es < 4, es el año anterior
DF_VOL_MES['WATER_YEAR'] = DF_VOL_MES.apply(
    lambda row: row['year'] if row['month'] >= 4 else row['year'] - 1,
    axis=1
)

# Sumar el volumen total por año hidrológico para las columnas seleccionadas
DF_VOL_ANUAL = DF_VOL_MES.groupby('WATER_YEAR')[cols].sum().reset_index()

HTML(DF_VOL_ANUAL.to_html())

,WATER_YEAR,80%_CG,80%_PS,TOMA AACC,NO REGULADO,TRASVASE,ARAZATI_1,ARAZATI_2,DEMANDA
0,1987.0,19.699776,44.402112,362.753280,298.668672,9.985248,9.04176,4.32432,63.954144
1,1988.0,36.818208,141.909984,541.012032,362.265984,40.050720,36.26640,17.34480,240.068448
2,1989.0,19.025280,56.157120,547.665120,472.533696,40.050720,36.26640,17.34480,240.068448
3,1990.0,55.166688,342.027936,1651.554432,1254.395808,40.050720,36.26640,17.34480,240.068448
4,1991.0,53.554176,567.705888,2540.808864,1919.540160,40.160448,36.36576,17.39232,240.781248
5,1992.0,43.067232,421.081632,2087.940960,1623.792096,40.050720,36.26640,17.34480,240.068448
6,1993.0,95.742432,744.708096,3546.289440,2705.812128,40.050720,36.26640,17.34480,240.068448
7,1994.0,61.336512,396.466560,1954.991520,1497.216096,40.050720,36.26640,17.34480,240.068448
8,1995.0,28.743264,373.314528,1470.516480,1068.458688,40.160448,36.36576,17.39232,240.781248
9,1996.0,23.347008,226.263744,947.121120,697.544064,40.050720,36.26640,17.34480,240.068448
